# Data augmentation

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import librosa
import matplotlib.pyplot as plt
import os
from scipy.io.wavfile import write
import cv2
import sounddevice as sd
import IPython.display as ipd
import noisereduce as nr

EPS = 1e-8


/home/incentive/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [ ]:
def get_spectrogram(wav):
    D = librosa.stft(wav, n_fft=480, hop_length=160,
                     win_length=480, window='hamming')
    spect, phase = librosa.magphase(D)
    return spect

In [ ]:
file_path = 'recordings/'+'0_tuli_2.wav'
wav, sr = librosa.load(file_path, sr=None)
print(wav.shape, wav.max(), wav.min())

In [ ]:
ipd.Audio(wav, rate=sr)

In [ ]:
def augmentation():
    for f in os.listdir('recordings/'):
        if f.endswith('.wav'):
            name = f.replace('.wav','')
            print(f)
            wav, sr = librosa.load('recordings/' + f)
            speed_rate = np.random.uniform(0.7,1.3)
            wav_speed_tune = cv2.resize(wav, (1, int(len(wav) * speed_rate))).squeeze()
            print('speed rate: %.3f' % speed_rate, '(lower is faster)')
            if len(wav_speed_tune) < 16000:
                pad_len = 16000 - len(wav_speed_tune)
                wav_speed_tune = np.r_[np.random.uniform(-0.001,0.001,int(pad_len/2)),
                           wav_speed_tune,
                           np.random.uniform(-0.001,0.001,int(np.ceil(pad_len/2)))]
            else: 
                cut_len = len(wav_speed_tune) - 16000
                wav_speed_tune = wav_speed_tune[int(cut_len/2):int(cut_len/2)+16000]
#             sd.play(wav,sr)
#             sd.wait(5)
            print('wav length: ', wav_speed_tune.shape[0])
            write('data collect/' +name+'_speed'+'.wav', sr,wav_speed_tune)
            
# augmentation()
            

In [23]:
def augmentation():
    for f in os.listdir('recordings/'):
        if f.endswith('.wav'):
            name = f.replace('.wav','')
            print(f)
            wav, sr = librosa.load('recordings/'+f)
            bg, sr = librosa.load('test record/'+ 'audio.wav')
            start_ = np.random.randint(bg.shape[0])
            bg_slice = bg[start_ : start_+16666]
            wav_with_bg = wav * np.random.uniform(0.8, 1.2) + \
              bg_slice * np.random.uniform(0, 0.1)
            noisy_part = wav_with_bg[10000:15000] 
            sd.play(wav_with_bg)
#             reduced_noise = nr.reduce_noise(audio_clip=wav_with_bg, noise_clip=noisy_part, verbose=False)
#             sd.play(reduced_noise,sr)
#             sd.wait(5)
#             ipd.Audio(wav_with_bg, rate=sr) 

            
            
augmentation()

8_atik_8.wav


ValueError: operands could not be broadcast together with shapes (11025,) (9759,) 

In [17]:
wav, sr = librosa.load('test record/'+'audio.wav')
sd.play(wav,sr)
wav.shape

(16538,)

In [18]:
wav, sr = librosa.load('recordings/'+'6_adeeb_7_speed.wav')
sd.play(wav,sr)
wav.shape

(16000,)

In [11]:
np.random.uniform(0.8, 1.2)

1.120081668138703

In [ ]:
import utils
from sklearn.model_selection import train_test_split
import numpy as np
import librosa
import os
from keras.utils import to_categorical
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
import keras
from sklearn.metrics import classification_report
from utils import wav2mfcc, model, get_data
from keras.utils import to_categorical



In [ ]:

def get_data():
    pad2d = lambda a, i: a[:, 0: i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0],i - a.shape[1]))))
    labels = []
    mfccs = []

    for f in os.listdir('recordings/'):
        if f.endswith('.wav'):
            wav, sr = librosa.load('recordings/' + f)
            mfcc = librosa.feature.mfcc(wav)
            padded_mfcc = pad2d(mfcc,40)
            mfccs.append(padded_mfcc)
            label = f.split('_')[0]
            labels.append(label)
    return np.array(mfccs), to_categorical(labels)


In [ ]:
#a,b = get_data()


In [ ]:

#b.shape,a.shape


In [ ]:
def get_cnn_model(input_shape, num_classes):
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())

    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(BatchNormalization())

    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

    return model


In [ ]:
def get_all():
    mfccs, labels = get_data()

    dim_1 = mfccs.shape[1]
    dim_2 = mfccs.shape[2]
    channels = 1
    classes = 10

    X = mfccs
    X = X.reshape((mfccs.shape[0], dim_1, dim_2, channels))
    y = labels

    input_shape = (dim_1, dim_2, channels)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

    model = get_cnn_model(input_shape, classes)

    return X_train, X_test, y_train, y_test,model

In [ ]:
X_train, X_test, y_train, y_test,cnn_model = get_all()


In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
cnn_model.summary()

In [ ]:
keras_callback = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=1,
                                             write_graph=True, write_images=True)

cnn_model.fit(X_train, y_train, batch_size=64, epochs=50, verbose=1, validation_split=0.1, callbacks=[keras_callback])

cnn_model.save('trained_model_new.h5')



In [ ]:
def check_preds(X, y):
    trained_model = keras.models.load_model('trained_model_new.h5')
    predictions = trained_model.predict_classes(X)

    print(classification_report(y, to_categorical(predictions)))

In [ ]:
check_preds(X_test, y_test)

In [ ]:
import sounddevice as sd
wav, sr = librosa.load('3_atik_34.wav')
sd.play(wav,sr)
pad2d = lambda a, i: a[:, 0: i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0],i - a.shape[1]))))
mfcc = librosa.feature.mfcc(wav)
padded_mfcc = pad2d(mfcc,40)
data = padded_mfcc.reshape(1,padded_mfcc.shape[0],padded_mfcc.shape[1],1)

In [ ]:
trained_model = keras.models.load_model('trained_model_new.h5')
predictions = trained_model.predict_classes(data)

In [ ]:
predictions

In [ ]:
y_test[:2,:]

In [ ]:
pad2d = lambda a, i: a[:, 0: i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0],i - a.shape[1]))))
import random as rn
random_file = rn.choice(os.listdir('recordings'))
print(random_file)
# mfccs.append(padded_mfcc)
wav, sr = librosa.load('recordings/' + '3_atik_31.wav')
mfcc = librosa.feature.mfcc(wav)
padded_mfcc = pad2d(mfcc,40)

In [ ]:
r = padded_mfcc.shape[0]
c  = padded_mfcc.shape[1]
r,c

In [ ]:
data = padded_mfcc.reshape(1,r,c,1)

In [ ]:
trained_model = keras.models.load_model('trained_model_new.h5')
predictions = trained_model.predict_classes(data)

In [ ]:
predictions

In [ ]:
pad2d = lambda a, i: a[:, 0: i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0],i - a.shape[1]))))
wav, sr = librosa.load('recordings/' + '8_atik_01.wav')
mfcc = librosa.feature.mfcc(wav)
padded_mfcc = pad2d(mfcc,40)

In [ ]:
data = padded_mfcc.reshape(1,padded_mfcc.shape[0],padded_mfcc.shape[1],1)

In [ ]:
#trained_model = keras.models.load_model('trained_model_new.h5')
predictions = trained_model.predict_classes(data)

In [ ]:
predictions

In [ ]:
import sounddevice as sd
sd.play(wav,24000)

In [ ]:
# import librosa
# import noisereduce as nr
# audio_data, sampling_rate = librosa.load('recordings/'+'temp.wav')
# noisy_part = audio_data[0:22000]  
# reduced_noise = nr.reduce_noise(audio_clip=audio_data, noise_clip=noisy_part, verbose=False)
# librosa.output.write_wav('8_atik_01.wav', reduced_noise, sampling_rate)

In [ ]:
wav, sr = librosa.load('recordings/'+'8_atik_01.wav')
sd.play(wav,sr)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(wav)

In [ ]:
import utils
from sklearn.model_selection import train_test_split
import numpy as np
import librosa
import os
from keras.utils import to_categorical
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
import keras
from sklearn.metrics import classification_report
from utils import wav2mfcc, model, get_data
from keras.utils import to_categorical


In [ ]:
import sounddevice as sd
wav, sr = librosa.load('recordings/'+'3_atik_31.wav')
sd.play(wav,sr)
pad2d = lambda a, i: a[:, 0: i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0],i - a.shape[1]))))
mfcc = librosa.feature.mfcc(wav)
padded_mfcc = pad2d(mfcc,40)
data = padded_mfcc.reshape(1,padded_mfcc.shape[0],padded_mfcc.shape[1],1)

In [ ]:
trained_model = keras.models.load_model('trained_model_new.h5')
predictions = trained_model.predict_classes(data)

In [ ]:
predictions